In [12]:
import os
import sys
import torch
import torch.nn
import argparse
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from networks.resnet import resnet50

# file_dir = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/datasets/DiffusionForensics/dalle2/celebahq/1_fake/100.png'
weights_path = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_98_epoch.pth'
# weights_path = "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/NPR.pth"
file_dir = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/datasets/DiffusionForensics/dalle2/celebahq/0_real/000152.jpg'

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-f','--file', default=file_dir)
parser.add_argument('-m','--model_path', type=str, default=weights_path)
parser.add_argument('-c','--crop', type=int, default=None, help='by default, do not crop. specify crop size')
parser.add_argument('--use_cpu', action='store_true', help='uses gpu by default, turn on to use cpu')

opt = parser.parse_args()

model = resnet50(num_classes=1)
state_dict = torch.load(opt.model_path, map_location='cpu')
model.load_state_dict(torch.load(opt.model_path, map_location='cuda:0'), strict=True)
if(not opt.use_cpu):
  model.cuda()
model.eval()

# Transform
trans_init = []
if(opt.crop is not None):
  trans_init = [transforms.CenterCrop(opt.crop),]
  print('Cropping to [%i]'%opt.crop)
else:
  print('Not cropping')
trans = transforms.Compose(trans_init + [
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = trans(Image.open(file_dir).convert('RGB'))

with torch.no_grad():
  in_tens = img.unsqueeze(0)
  if not opt.use_cpu:
    in_tens = in_tens.cuda()
  prob = model(in_tens).sigmoid().item()

print('probability of being synthetic: {:.2f}%'.format(prob * 100))


/tmp/ipykernel_1605245/4278691843.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(opt.model_path, map_location='cpu')
/tmp/ipykernel_1605245/427

Not cropping
probability of being synthetic: 0.00%


In [11]:
import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np

class ImageProcessor:
    def interpolate(self, img, factor):
        # Двойная интерполяция с уменьшением и увеличением
        img_downscaled = F.interpolate(img, scale_factor=factor, mode='nearest', recompute_scale_factor=True)
        img_upscaled = F.interpolate(img_downscaled, scale_factor=1/factor, mode='nearest', recompute_scale_factor=True)
        return img_upscaled

    def apply_npr(self, img):
        # NPR = x - self.interpolate(x, 0.5)
        interpolated_img = self.interpolate(img, 0.5)
        npr_result = img - interpolated_img
        return npr_result

def save_image(tensor, filename):
    # Преобразуем тензор обратно в изображение и сохраняем его
    tensor = tensor.squeeze()  # Удаляем лишние размеры, если есть
    tensor = tensor.permute(1, 2, 0).cpu().numpy()  # Перемещаем каналы для PIL
    tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min())  # Нормализуем для отображения в 0-1
    img = Image.fromarray((tensor * 255).astype(np.uint8))
    img.save(filename)

# Пример использования
# Загружаем изображение и преобразуем его в тензор
file_dir = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/datasets/DiffusionForensics/dalle2/celebahq/0_real/000152.jpg'
img = Image.open(file_dir)
img = img.convert('RGB')  # Преобразуем в RGB, если не RGB
img_tensor = torch.from_numpy(np.array(img)).float().permute(2, 0, 1).unsqueeze(0)  # Преобразуем в тензор, [B, C, H, W]

# Применяем операцию NPR
processor = ImageProcessor()
npr_image = processor.apply_npr(img_tensor)

# Сохраняем результат
save_image(npr_image, 'npr_result.jpg')
